In [1]:
import pandas as pd
import numpy as np
from collections import Counter
import tensorflow as tf
import tflearn
from tflearn.data_utils import to_categorical

curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [2]:
from sklearn.utils import shuffle

In [3]:
trainX = pd.read_csv('train_MLWARE1.csv')
test = pd.read_csv('test_MLWARE1.csv')

In [4]:
train = shuffle(trainX, random_state=25)

In [5]:
train = train.reset_index()

In [6]:
train.head(10)

,index,ID,tweet,label
0,21590,T000473948,@beasangel you're you're new to the internet a...,sarcastic
1,48598,T000500956,b'If congress wanted to make it simpler they w...,sarcastic
2,44788,T000497146,"#sarcasm sorry thought you'd catch it""",sarcastic
3,5541,T000457899,b'the changes made to the sat totally undermin...,sarcastic
4,58456,T000510814,b'just dance was just announced on i love the ...,non-sarcastic
5,18773,T000471131,really love it when niggas sweat them #not me',sarcastic
6,61927,T000514285,drinks delicious steak salad best company love...,non-sarcastic
7,6713,T000459071,b'Los de en Sambueza de verdad Y con Boy mezcl...,sarcastic
8,52234,T000504592,b'rt turning left onto steeles ',non-sarcastic
9,8370,T000460728,b'Wow glad my alarm is going off in hours $$SA...,sarcastic


In [7]:
test.head(2)

,ID,tweet
0,T000543656,and baby showers the new weddings lmao'
1,T000543657,in only white people can be racist if a white ...


In [8]:
tweets = pd.DataFrame(train.tweet)
tweets.tweet[0]

"@beasangel you're you're new to the internet aren't you"

In [9]:
len(tweets)

91298

In [10]:
train.shape

(91298, 4)

# COUNTING word frequency

In [11]:
total_counts = Counter()
saracstic_counts = Counter()
non_sarcastic_counts = Counter()

for word in train.tweet.values:
    total_counts.update(word.split(' '))
    
print(len(total_counts))

57562


In [12]:
total_counts.most_common(10)

[('to', 31931),
 ('the', 29937),
 ('a', 21607),
 ('and', 14796),
 ('for', 14677),
 ('you', 14186),
 ('my', 13507),
 ('of', 13479),
 ('is', 13208),
 ('in', 12667)]

In [13]:
vocab = sorted(total_counts, key= total_counts.get , reverse = True)[:8000]
print(vocab[-1], total_counts[vocab[-1]])

recall 10


In [14]:
word2idx = {word :i for i, word in enumerate(vocab)}

In [15]:
word2idx

{'pole': 6002,
 'tuesday': 1200,
 'display': 3703,
 "b'happy": 265,
 'females': 5958,
 'visit': 649,
 'cloud': 5399,
 'hill': 4298,
 'shade': 7542,
 "b'Dear": 3312,
 'quizzes': 7841,
 'b"Oh': 466,
 'storage': 7636,
 'parts': 3135,
 'guarantee': 6643,
 "b'found": 4118,
 "b'check": 1188,
 'Having': 4672,
 'COOL': 5855,
 'jk': 5775,
 'chance': 837,
 'lazy': 2807,
 'Like': 1278,
 'Bryan': 7842,
 'down': 194,
 'estaba': 7758,
 'charming': 4828,
 'period': 2224,
 'italy': 3432,
 'psychology': 5947,
 'obama': 4373,
 "b'no": 1121,
 "fun'": 2795,
 'kidding': 1453,
 'WO': 6557,
 'bachelor': 7041,
 'euro': 3091,
 'Ben': 3945,
 'Awesome': 1834,
 'aggressive': 5363,
 'lift': 3251,
 'legal': 2184,
 'close': 674,
 'Where': 2265,
 'bridal': 6047,
 "b'damn": 6291,
 'program': 2213,
 'afraid': 2124,
 "good'": 2711,
 'care': 345,
 'pants': 1754,
 'stepped': 7694,
 'garden': 1594,
 'concert': 881,
 'b"Ya': 5812,
 'jerseys': 6142,
 'perfect': 305,
 'liar': 4405,
 'E': 5774,
 'Kim': 4797,
 "b'Apparently": 3

In [16]:
print(len(vocab))

8000


In [17]:
def text_to_vector(text):
    word2vec = np.zeros(len(vocab) , dtype = np.int_)
    
    for word in text.split(' '):
        idx = word2idx.get(word , None)
        if idx is None:
            continue
        else:
            word2vec[idx] = 1
            
    return np.array(word2vec)

In [18]:
text_to_vector('the motion to the sales are in  a motion')

array([1, 1, 1, ..., 0, 0, 0])

In [19]:
word_vectors = np.zeros( (len(tweets),len(vocab)), dtype = np.int_)

In [20]:
word_vectors.shape

(91298, 8000)

In [21]:
for i, (_, word) in enumerate(tweets.iterrows()):
    word_vectors[i] = text_to_vector(word[0])

In [22]:
word_vectors[:5, :20]

array([[1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0]])

In [23]:
labels = train.label

In [24]:
Y = (labels=='sarcastic').astype(np.int_)

In [25]:
test_tweets = pd.DataFrame(test.tweet)
test_word2vectors = np.zeros( (len(test_tweets),len(vocab)), dtype = np.int_)
for i, (_, word) in enumerate(test_tweets.iterrows()):
    test_word2vectors[i] = text_to_vector(word[0])

In [26]:
test_word2vectors[0]

array([0, 1, 0, ..., 0, 0, 0])

In [27]:
#from sklearn.model_selection import train_test_split

In [28]:
#records = len(labels)
#shuffle = np.arange(records)
#np.random.shuffle(shuffle)

In [29]:
#print(len(shuffle),len(word_vectors))

In [30]:
#shuffle

In [31]:
#trainY = y[shuffle,:]

In [32]:
# X_train, X_test, y_train, y_test = train_test_split(word_vectors, y, test_size=0.10, random_state=42)

In [27]:
Y[0]

1

## BUILD MODEL

In [34]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier


In [28]:
from sklearn.dummy import DummyClassifier

In [29]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import KFold, learning_curve
from sklearn.metrics import f1_score

In [30]:
import warnings
warnings.filterwarnings('ignore')

In [51]:
from sklearn import svm

In [52]:
clf = svm.SVC()

In [53]:
clf.fit(word_vectors, Y)

MemoryError: 

In [ ]:
clf.score(test_word_vectors, y_test) 

In [40]:
Yac_test = clf.predict(test_word2vectors)

In [41]:
Yac_test

array([0, 1, 1, ..., 1, 1, 1])

In [42]:
pred = []

In [43]:
for i in range(len(Yac_test)):
    if(Yac_test[i]>0.5):
        pred.append('sarcastic')
    else:
        pred.append('non-sarcastic')

In [44]:
len(pred)

39392

In [45]:
pred = pd.DataFrame(pred)
pred['ID']=test.ID

In [46]:
cols=[1,0]
pred.columns = ['ID','label']

In [47]:
pred = pred[cols]

In [48]:
pred.columns = ['ID','label']

In [49]:
pred

,ID,label
0,T000543656,non-sarcastic
1,T000543657,sarcastic
2,T000543658,sarcastic
3,T000543659,non-sarcastic
4,T000543660,sarcastic
5,T000543661,sarcastic
6,T000543662,sarcastic
7,T000543663,sarcastic
8,T000543664,sarcastic
9,T000543665,sarcastic


In [50]:
pred.to_csv('Random.csv',index=False)